In [3]:
from IPython.display import display, Math
import numpy as np
import pandas as pd
import sympy as sp
from sympy import symbols, sympify, Matrix, Eq, solve, expand, Abs, pprint, factor
from itertools import permutations
import scipy

# Attempt at Coding Algorithm A

The only variables you need to toy around with are sig and pi below


In [2]:
def contingency_tables(row_sums, col_sums):
    '''
    This function generates the contingency tables
    for a given set of row and column sums
    using a backtracking/recursive approach.
    '''
    m, n = len(row_sums), len(col_sums)
    result = []

    def backtrack(row, table, col_sums_left):
        if row == m:
            if all(c == 0 for c in col_sums_left):
                result.append(np.array(table))
            return

        def valid_next_rows(rsum, cols_left, partial=[]):
            if len(partial) == n:
                if sum(partial) == rsum:
                    yield list(partial)
                return
            i = len(partial)
            max_entry = min(rsum - sum(partial), cols_left[i])
            for x in range(max_entry + 1):
                yield from valid_next_rows(rsum, cols_left, partial + [x])

        for row_vals in valid_next_rows(row_sums[row], col_sums_left):
            new_cols = [c - x for c, x in zip(col_sums_left, row_vals)]
            if all(c >= 0 for c in new_cols):
                backtrack(row + 1, table + [row_vals], new_cols)

    backtrack(0, [], col_sums)
    return result

In [3]:
def generate_monomials(tables, pi, sig):
    '''
    This function simply fills out the above dictionaries and lists
    @params tables is a list of contingency tables generated by the above function
    '''
    m, n = len(sig), len(pi)

    # Create symbols z_ij for monomial basis
    z = Matrix(m, n, lambda i, j: symbols(f"z{i+1}{j+1}"))

    monomials = []  # monomials[i] returns the monomial generated for the ith contingency table
    idx_dict = {} # idx_dict[i] returns the sequence of bump chains to be inserted for the ith contingency table
    exp_dict = {} # exp_dict[i] returns a a dictionary of monomials paired with their respective exponent for the ith contingency table

    idx = 0 # dummy counter
    for table in tables:
        mon = 1
        temp_idx = []
        temp_exp_idx = []
        table_dict = {}
        for j in range(m):
            for i in range(n):
                exp = table[i][j]
                if exp > 0:
                    pair = (i + 1, j + 1)
                    temp_idx.extend([pair] * exp)
                    mon *= z[i, j]**exp
                    if z[i, j] not in table_dict:
                        table_dict[z[i, j]] = exp
                    else:
                        table_dict[z[i, j]] += exp
        exp_dict[idx] = table_dict
        idx_dict[idx] = temp_idx

        idx += 1
        monomials.append(mon)

    return monomials, idx_dict, exp_dict

In [4]:
def rsk_insert(tableau, x):
    tableau = tableau.copy()
    rows, cols = tableau.shape
    bumped = x
    for r in range(rows):
        row = tableau[r]
        mask = (row > 0)
        eligible = row[mask]
        idx = np.where(eligible > bumped)[0]
        if idx.size == 0:
            insert_pos = np.sum(mask)
            if insert_pos < cols:
                tableau[r, insert_pos] = bumped
                return tableau, (r, insert_pos)
            else:
                continue
        else:
            i = idx[0]
            bumped, tableau[r, i] = tableau[r, i], bumped
    empty_row = np.zeros(cols, dtype=int)
    empty_row[0] = bumped
    tableau = np.vstack([tableau, empty_row])
    return tableau, (tableau.shape[0] - 1, 0)

def viennot_rsk(biword):
    n = len(biword)
    P = np.zeros((n, n), dtype=int)
    Q = np.zeros((n, n), dtype=int)
    for a, b in biword:
        P, (r, c) = rsk_insert(P, a)
        Q[r, c] = b
    return P, Q

def print_tableau(tableau, name='T'):
    '''
    for debugging
    @params tableau is a numpy array
    @params name is a string
    '''
    print(f"{name}:")
    for row in tableau:
        row_nonzero = row[row > 0]
        if row_nonzero.size > 0:
            print(row_nonzero)

In [5]:
def determinant(P, Q, idx):
  '''
  This function generates the determinant
  for the given RSK correspondence tableaus
  P and Q. @params idx refers to the ith
  contingency table.
  '''
  i = 0
  det_collection = []

  for col_idx in range(P.shape[1]):
      n = np.count_nonzero(P[:, col_idx])
      p_col = P[:, col_idx]
      q_col = Q[:, col_idx]
      det = Matrix.zeros(n,n)
      for i, p_val in enumerate(p_col):
        for j, q_val in enumerate(q_col):
          if p_val != 0 and q_val != 0:
            key = z[p_val - 1, q_val - 1]
            det[i, j] = key
      det_collection.append(det)

  expr = 1
  for det in det_collection:
    minor_det = det.det()
    expr *= minor_det
  expr = expand(expr)
  return expr

In [6]:
def generate_final_matrix(monomial_mapping, determinant_list):
    '''
    This block of code uses the initial
    dictionary to fill out the final matrix
    '''

    final_matrix = np.zeros((len(monomial_mapping), len(monomial_mapping)))
    final_matrix

    final_size = len(monomial_mapping)
    for col_idx in range(final_size):
        curr_list = determinant_list[col_idx]
        final_slice = final_matrix[:, col_idx]

        for term in curr_list.as_ordered_terms():
          coeff, monomial = term.as_coeff_Mul()

          if monomial in monomial_mapping:
              row_idx = monomial_mapping[monomial]
              final_slice[row_idx] = coeff

          elif -monomial in monomial_mapping:
              row_idx = monomial_mapping[-monomial]
              final_slice[row_idx] = -coeff

        final_matrix[:, col_idx] = final_slice
    return final_matrix

In [7]:
def foo(matrix):
        '''
        function that may be a bit faster than np.linalg.eig()
        @params matrix is a numpy array
        '''
        eigenvalues, eigenvectors = np.linalg.eig(matrix)
        for eigenvalue in np.unique(eigenvalues):
            algebraic_multiplicity = np.sum(eigenvalues == eigenvalue)

            eigenvectors_for_eigenvalue = eigenvectors[:, eigenvalues == eigenvalue]
            geometric_multiplicity = np.linalg.matrix_rank(eigenvectors_for_eigenvalue)

            if algebraic_multiplicity != geometric_multiplicity:
                return False
        return True

def display(final_matrix):
    print(f"Sigma: {sig}")
    print(f"Pi: {pi}")
    print(f"Trace: {np.linalg.trace(final_matrix)}")
    print(f"is Diagonalizable: {foo(final_matrix)}" )
    print(f"Determinant: {np.linalg.det(final_matrix)}")
    char_poly_coeffs = np.poly(final_matrix)
    x = symbols('x')
    num_char_poly_coeffs = len(char_poly_coeffs)
    char_poly = [char_poly_coeffs[n] * x**(num_char_poly_coeffs - n - 1) for n in range(num_char_poly_coeffs)]
    pretty_eq = sp.latex(factor(sum(char_poly)))
    pprint(f"Characteristic Polynomial:")
    print(display(Math(pretty_eq)))

<!-- end of fxn code--!>

In [23]:
from IPython.display import display, Math

sig = [7, 8, 2]
pi = [3, 5, 9]
m, n = len(sig), len(pi)

# Create symbols z_ij for monomial basis
z = Matrix(m, n, lambda i, j: symbols(f"z{i+1}{j+1}"))
# table sanity check
tables = contingency_tables(sig, pi)
print(f"Found {len(tables)} tables.")
tables = tables[::-1]
for t in tables:
    print(t, end="\n\n")
# gives the correct hashing order for the ordered monomial basis
monomials, idx_dict, exp_dict = generate_monomials(tables, pi, sig)
monomial_mapping = {mon: i for i, mon in enumerate(monomials)}; monomial_mapping
numpy_tableaus = [viennot_rsk(index) for index in idx_dict.values()] # converting our bump chains(?) into Tableaus P and Q
determinant_list = [determinant(P, Q, i) for i, (P, Q) in enumerate(numpy_tableaus)] # determinants generated from P,Q

final_matrix = generate_final_matrix(monomial_mapping, determinant_list)
print(final_matrix)
def foo(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    for eigenvalue in np.unique(eigenvalues):
        algebraic_multiplicity = np.sum(eigenvalues == eigenvalue)

        eigenvectors_for_eigenvalue = eigenvectors[:, eigenvalues == eigenvalue]
        geometric_multiplicity = np.linalg.matrix_rank(eigenvectors_for_eigenvalue)

        if algebraic_multiplicity != geometric_multiplicity:
            return False
    return True
print(f"Sigma: {sig}")
print(f"Pi: {pi}")
print(f"Trace: {np.linalg.trace(final_matrix)}")
print(f"is Diagonalizable: {foo(final_matrix)}" )
print(f"Determinant: {np.linalg.det(final_matrix)}")
# print(f"Determinant: {np.linalg.det(final_matrix)}")
char_poly_coeffs = np.poly(final_matrix)
x = symbols('x')
num_char_poly_coeffs = len(char_poly_coeffs)
char_poly = [char_poly_coeffs[n] * x**(num_char_poly_coeffs - n - 1) for n in range(num_char_poly_coeffs)]
pretty_eq = sp.latex(factor(sum(char_poly)))
pprint(f"Characteristic Polynomial:")
print(display(Math(pretty_eq)))

Found 104 tables.
[[3 4 0]
 [0 1 7]
 [0 0 2]]

[[3 4 0]
 [0 0 8]
 [0 1 1]]

[[3 3 1]
 [0 2 6]
 [0 0 2]]

[[3 3 1]
 [0 1 7]
 [0 1 1]]

[[3 3 1]
 [0 0 8]
 [0 2 0]]

[[3 2 2]
 [0 3 5]
 [0 0 2]]

[[3 2 2]
 [0 2 6]
 [0 1 1]]

[[3 2 2]
 [0 1 7]
 [0 2 0]]

[[3 1 3]
 [0 4 4]
 [0 0 2]]

[[3 1 3]
 [0 3 5]
 [0 1 1]]

[[3 1 3]
 [0 2 6]
 [0 2 0]]

[[3 0 4]
 [0 5 3]
 [0 0 2]]

[[3 0 4]
 [0 4 4]
 [0 1 1]]

[[3 0 4]
 [0 3 5]
 [0 2 0]]

[[2 5 0]
 [1 0 7]
 [0 0 2]]

[[2 5 0]
 [0 0 8]
 [1 0 1]]

[[2 4 1]
 [1 1 6]
 [0 0 2]]

[[2 4 1]
 [1 0 7]
 [0 1 1]]

[[2 4 1]
 [0 1 7]
 [1 0 1]]

[[2 4 1]
 [0 0 8]
 [1 1 0]]

[[2 3 2]
 [1 2 5]
 [0 0 2]]

[[2 3 2]
 [1 1 6]
 [0 1 1]]

[[2 3 2]
 [1 0 7]
 [0 2 0]]

[[2 3 2]
 [0 2 6]
 [1 0 1]]

[[2 3 2]
 [0 1 7]
 [1 1 0]]

[[2 2 3]
 [1 3 4]
 [0 0 2]]

[[2 2 3]
 [1 2 5]
 [0 1 1]]

[[2 2 3]
 [1 1 6]
 [0 2 0]]

[[2 2 3]
 [0 3 5]
 [1 0 1]]

[[2 2 3]
 [0 2 6]
 [1 1 0]]

[[2 1 4]
 [1 4 3]
 [0 0 2]]

[[2 1 4]
 [1 3 4]
 [0 1 1]]

[[2 1 4]
 [1 2 5]
 [0 2 0]]

[[2 1 4]
 [0 4 4]
 [1 0 1

<IPython.core.display.Math object>

None


# V1 code for RSK_{m,n,d}:
- @param m,n: len of pi/sig weight vectors
- @param d: degree

In [9]:
def contingency_vectors(n, d):
    '''
    This function generates the possible contingency vectors
    @params n: len of vector
    @params d: degree
    '''
    vectors = []
    def recusion(vector, degrees_left):
        if len(vector) == n:
            vectors.append(vector)
            return
        for i in range(degrees_left + 1)[::-1]:
            new_vector = vector + [i]

            if len(new_vector) == n:  # pruning
                vectors.append(new_vector)
                return

            recusion(new_vector, degrees_left - int(i))

    recusion([], d)
    return vectors

contingency_vectors(3, 7)

[[7, 0, 0],
 [6, 1, 0],
 [6, 0, 1],
 [5, 2, 0],
 [5, 1, 1],
 [5, 0, 2],
 [4, 3, 0],
 [4, 2, 1],
 [4, 1, 2],
 [4, 0, 3],
 [3, 4, 0],
 [3, 3, 1],
 [3, 2, 2],
 [3, 1, 3],
 [3, 0, 4],
 [2, 5, 0],
 [2, 4, 1],
 [2, 3, 2],
 [2, 2, 3],
 [2, 1, 4],
 [2, 0, 5],
 [1, 6, 0],
 [1, 5, 1],
 [1, 4, 2],
 [1, 3, 3],
 [1, 2, 4],
 [1, 1, 5],
 [1, 0, 6],
 [0, 7, 0],
 [0, 6, 1],
 [0, 5, 2],
 [0, 4, 3],
 [0, 3, 4],
 [0, 2, 5],
 [0, 1, 6],
 [0, 0, 7]]

In [10]:
def generate_matrices(m, n, d):
    '''
    This function generates the RSK_{m,n,d} matrices
    @params m,n: len of pi/sig weight vectors can assume m = n
    @params d: degree
    '''
    contingency_vectors_list = contingency_vectors(m, d)
    contingency_vector_map = {i : tuple(vector) for i, vector in enumerate(contingency_vectors_list)}
    matrices = []
    final_dim = 0
    for pi in contingency_vector_map.values():
        for sig in contingency_vector_map.values():
            tables = contingency_tables(sig, pi)
            monomials, idx_dict, exp_dict = generate_monomials(tables, pi, sig)
            numpy_tableaus = [viennot_rsk(index) for index in idx_dict.values()]
            determinant_list = [determinant(P, Q, i) for i, (P, Q) in enumerate(numpy_tableaus)]
            monomial_mapping = {mon: i for i, mon in enumerate(monomials)}; monomial_mapping
            final_matrix = generate_final_matrix(monomial_mapping, determinant_list)
            final_dim += final_matrix.shape[0]
            matrices.append(final_matrix)
    return matrices

In [11]:
from scipy.linalg import block_diag
m = 3
d = 2
matrices = generate_matrices(m,m,d)
final_rsk_matrix = block_diag(*matrices)

In [12]:
from scipy.linalg import block_diag
from IPython.display import display, Math
'''
Edit m and d below to obtain the desired RSK_{m,m,d} matrix
'''
m = 3
d = 3
matrices = generate_matrices(m,m,d)
final_rsk_matrix = block_diag(*matrices)

def foo(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    for eigenvalue in np.unique(eigenvalues):
        algebraic_multiplicity = np.sum(eigenvalues == eigenvalue)

        eigenvectors_for_eigenvalue = eigenvectors[:, eigenvalues == eigenvalue]
        geometric_multiplicity = np.linalg.matrix_rank(eigenvectors_for_eigenvalue)

        if algebraic_multiplicity != geometric_multiplicity:
            return False
    return True
print(f"m: {m}")
print(f"n: {n}")
print(f"d: {d}")
print(f"Trace: {np.linalg.trace(final_rsk_matrix)}")
print(f"Determinant: {np.linalg.det(final_rsk_matrix)}")

m: 3
n: 3
d: 3
Trace: 42.0
Determinant: -1.0


In [13]:
# check_table = []
# trace_table = []
# for i in range(4):
#   temp_row_trace = []
#   temp_row_det = []
#   for d in range(4):
#     matrices = generate_matrices(i+1,i+1,d+1)
#     final_rsk_matrix = block_diag(*matrices)
#     trace = np.linalg.trace(final_rsk_matrix)
#     det = np.linalg.det(final_rsk_matrix)
#     temp_row_trace.append(trace)
#     temp_row_det.append(det)
#   check_table.append(temp_row_trace)
#   trace_table.append(temp_row_det)
# print(np.array(check_table))
# print(np.array(trace_table))

m: 3
n: 5
d: 9
Trace: 660.0
Determinant: 0.9999999999999918


m = 3:
*   d = 8
  *   1m
  *   1
  *   270
*   d = 9
  *   5m
  *   1
  *   660

*   d = 10
  *

*   List item



Scratch Work for Algorithm 7.1

In [50]:
def naive_gen_reduced_pair(sig, pi):
  '''
  Left the naive algorithm in to help debug/easier to think about better soln
  @param pi = pi vector
  @param sig = sig vector
  fxn computes the reduced pair given unreduced (or reduced) pi/sig
  '''
  d_pi = len(np.where(pi != 0)[0])
  d_sig = len(np.where(sig != 0)[0])
  d = max(d_pi, d_sig) # degree

  # corollary 3.19 base cases
  # I
  if min(d_pi, d_sig) <= 1:
    return (np.zeros(d_pi), np.zeros(d_sig))
  # II
  if d_pi == 2 and d_sig == 2:
    abs_min_ele = min(min(pi), min(sig))
    reduced_ele = [abs_min_ele] * 2
    return (reduced_ele, reduced_ele)
  # III
  if len(np.where(sig != 0)[0]) == 2 and sig[1] == 1:
    red_sig = [max(len(sig), len(pi)) - 1, 1] # max wrapping for safety
    red_pi = [1] * d_pi
    return (red_sig, red_pi)
  # other cases
  def naive_greedy_reduction():
    while (max(pi) + max(sig)) > d:
        i = np.argmax(pi)
        j = np.argmax(sig)

        pi[i] -= 1
        sig[j] -= 1
        print(sig, pi)
    return (sig, pi)

  return naive_greedy_reduction()


In [52]:
sig = np.array([6, 1, 0])
pi = np.array([1, 1, 5])
print(naive_gen_reduced_pair(sig, pi))

([2, 1], [1, 1, 1])


In [16]:
import heapq

def gen_reduced_pair(sig, pi):
  pi_heap = [(-pi[i], i) for i in range(len(pi))]
  sig_heap = [(-sig[i], i) for i in range(len(sig))]
  d = max(len(pi), len(sig))

  heapq.heapify(pi_heap)
  heapq.heapify(sig_heap)
  print(pi_heap[0][0])
  while -pi_heap[0][0] - sig_heap[0][0] > d:
    pi_val, pi_idx = heapq.heappop(pi_heap)
    sig_val, sig_idx = heapq.heappop(sig_heap)

    pi[pi_idx] -= 1
    sig[sig_idx] -= 1
    heapq.heappush(pi_heap, (-pi[pi_idx], pi_idx))
    heapq.heappush(sig_heap, (-sig[sig_idx], sig_idx))

  return (sig, pi)


In [17]:
sig = [6, 1, 0]
pi = [1, 1, 5]
print(gen_reduced_pair(sig, pi))

-5
([2, 1, 0], [1, 1, 1])


In [27]:
pi = np.array([7, 8, 2])
sig = np.array([5, 3, 9])
print(gen_reduced_pair(sig, pi))

(array([4, 3, 3]), array([6, 2, 2]))


In [15]:
def determinant(P, Q, idx):
  '''
  This function generates the determinant
  for the given RSK correspondence tableaus
  P and Q. @params idx refers to the ith
  contingency table.
  '''
  det_collection = []

  for col_idx in range(P.shape[1]):
      # Find the indices where both P and Q have non-zero elements in this column
      non_zero_indices = np.nonzero((P[:, col_idx] != 0) & (Q[:, col_idx] != 0))[0]
      n_minor = len(non_zero_indices) # Size of the minor matrix

      # If there are no aligned non-zero elements, the determinant is 1 (empty product)
      if n_minor == 0:
          det_collection.append(sp.Integer(1)) # Use SymPy Integer 1
          continue

      det = Matrix.zeros(n_minor, n_minor) # Create the minor matrix with the correct size

      # Populate the minor matrix with corresponding z values
      for minor_row_idx, p_idx in enumerate(non_zero_indices):
          for minor_col_idx, q_idx in enumerate(non_zero_indices):
              p_val = P[p_idx, col_idx]
              q_val = Q[q_idx, col_idx]
              # Ensure indices are within the bounds of the z matrix
              if 0 <= p_val - 1 < z.shape[0] and 0 <= q_val - 1 < z.shape[1]:
                 key = z[p_val - 1, q_val - 1]
                 det[minor_row_idx, minor_col_idx] = key
              else:
                 # This case should ideally not happen if P and Q values are derived correctly from contingency tables
                 # If it does, it indicates an issue earlier in the process.
                 # For now, set the entry to 0 or handle as an error
                 det[minor_row_idx, minor_col_idx] = 0 # Or handle error appropriately

      try:
        minor_det = det.det()
      except Exception as e:
          print(f"Error computing determinant for column {col_idx}, minor matrix size {n_minor}: {det}")
          raise e # Re-raise the exception after printing information

      det_collection.append(minor_det)

  expr = 1
  for det_val in det_collection:
    expr *= det_val
  expr = expand(expr)
  return expr

In [16]:
def fast_det_rsk(contingency_vecs):
  ''' function for thm 7.1
  @param contingency_vecs = list of contingency vectors
  '''
  reduced_pairs_dict = {}

  true_det = 1

  for pi in contingency_vecs:
    for sig in contingency_vecs:
        r_sig, r_pi = naive_gen_reduced_pair(sig, pi)
        r_pair = (r_sig, r_pi)
        if r_pair in reduced_pairs_dict:
            true_det *= reduced_pairs_dict[r_pair]
        else:
            P, Q = viennot_rsk(r_pair)
            det = determinant(P, Q, 0)
            true_det *= det
            reduced_pairs_dict[r_pair] = det
  return true_det

In [ ]:
def fast_trace(contingency_vecs):
  ''' function for thm 7.1
  @param contingency_vecs = list of contingency vectors
  '''

